# ✂️ Chunking Demo (extended)

Цей ноутбук показує, як працює чанкінг (розбиття тексту на фрагменти) перед індексацією RAG.

**Що робимо:**
- завантажуємо файли з `data/raw/`
- розбиваємо тексти на чанки з overlap
- виводимо перші чанки + метадані
- аналізуємо розміри чанків і overlap
- перевіряємо сміття типу `=====` у чанках
- тестуємо простий пошук по ключових словах (lexical) для sanity-check


In [ ]:
# Якщо запускаєш ноутбук з папки notebooks/, треба додати корінь проєкту в sys.path
import sys
from pathlib import Path

ROOT = Path('..').resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print('Project root:', ROOT)


In [ ]:
from src.data_loader import load_raw_docs
from src.chunking import chunk_docs

docs = load_raw_docs('data/raw')
chunks = chunk_docs(docs, chunk_size=300, chunk_overlap=60)

print(f'Documents loaded: {len(docs)}')
print(f'Chunks created: {len(chunks)}')
print('Example doc meta:', getattr(docs[0], 'meta', None) if docs else None)


In [ ]:
# Подивимось перші 5 чанків
for i, ch in enumerate(chunks[:5], start=1):
    print('\n' + '='*60)
    print(f'Chunk #{i}')
    print('Meta:', ch.meta)
    print('-'*60)
    print(ch.text)


## 📏 Аналіз чанків

Подивимось:
- довжини чанків
- розподіл по документах
- чи є дуже короткі/дуже довгі чанки


In [ ]:
import re
from collections import defaultdict, Counter

def chunk_len(ch):
    return len(ch.text or '')

lens = [chunk_len(c) for c in chunks]
if lens:
    print('Min:', min(lens), 'Max:', max(lens), 'Avg:', round(sum(lens)/len(lens), 1))

    # buckets
    buckets = Counter()
    for L in lens:
        if L < 120: buckets['<120'] += 1
        elif L < 200: buckets['120-199'] += 1
        elif L < 300: buckets['200-299'] += 1
        elif L < 420: buckets['300-419'] += 1
        else: buckets['>=420'] += 1
    print('Length buckets:', dict(buckets))
else:
    print('No chunks')

# chunks per source
by_src = defaultdict(int)
for c in chunks:
    src = (c.meta or {}).get('source', 'unknown')
    by_src[src] += 1

print('\nChunks per source (top 20):')
for src, n in sorted(by_src.items(), key=lambda x: x[1], reverse=True)[:20]:
    print(f'  {src}: {n}')


## 🔁 Перевірка overlap (на рівні тексту)

Перевіримо: чи реально сусідні чанки мають спільний «хвіст/голову» приблизно на `chunk_overlap` символів.

> Це sanity-check, бо реалізація overlap може бути по символах/словах — залежить від `src.chunking`.

In [ ]:
def common_prefix_len(a: str, b: str, max_check=200):
    a = a or ''
    b = b or ''
    m = min(len(a), len(b), max_check)
    i = 0
    while i < m and a[i] == b[i]:
        i += 1
    return i

def common_suffix_prefix(suffix_from: str, prefix_from: str, max_check=200):
    s = suffix_from or ''
    p = prefix_from or ''
    m = min(len(s), len(p), max_check)
    # check from 1..m the largest k such that s[-k:] == p[:k]
    best = 0
    for k in range(1, m+1):
        if s[-k:] == p[:k]:
            best = k
    return best

# group by source and sort by start index if available
by_source = defaultdict(list)
for c in chunks:
    src = (c.meta or {}).get('source', 'unknown')
    by_source[src].append(c)

def start_pos(c):
    return (c.meta or {}).get('start', 0)

samples = 0
for src, arr in by_source.items():
    arr = sorted(arr, key=start_pos)
    for i in range(len(arr)-1):
        a, b = arr[i], arr[i+1]
        ov = common_suffix_prefix(a.text, b.text, max_check=250)
        if ov > 0:
            print(f"{src}: chunk {i} -> {i+1} overlap_like={ov}")
            samples += 1
        if samples >= 15:
            break
    if samples >= 15:
        break

if samples == 0:
    print('No detectable string overlap in first samples (може бути токенний/словний overlap або інша реалізація).')


## 🧽 Пошук «сміття» в чанках

Перевіряємо, чи присутні лінії типу:
- `=====`
- `-----`
- `____`
- `***`

Таке сміття потім може «вилазити» в відповідях.

In [ ]:
SEP_RE = re.compile(r"^(={3,}|-{3,}|_{3,}|\*{3,})\s*$", re.MULTILINE)

bad_chunks = []
for idx, ch in enumerate(chunks):
    if SEP_RE.search(ch.text or ''):
        bad_chunks.append(idx)

print('Chunks with separators:', len(bad_chunks))
print('First indexes:', bad_chunks[:20])

if bad_chunks:
    i = bad_chunks[0]
    print('\nExample bad chunk index:', i)
    print('Meta:', chunks[i].meta)
    print(chunks[i].text[:700])


## 🔎 Простий lexical-search по чанках (sanity check)

Це не заміна ембедінгів/RAG, але допомагає:
- швидко зрозуміти, чи ключові слова реально знаходяться в чанках
- подивитись топ-результати


In [ ]:
UA_STOP = {
  "що","це","та","і","й","або","але","в","у","на","до","від","для","про","як",
  "які","яка","яке","якій","якого","якою","яким","якими","чи","не","так","є",
  "з","із","за","над","під","при","між","о","об","по","коли","де","хто","ти","ви",
  "мені","тобі","йому","їй","їм","нас","вам","їх","тут","там","цей","ця","це","ці"
}

def normalize(s: str):
    s = (s or '').lower()
    s = s.replace('’', "'").replace('`', "'")
    s = re.sub(r"[^\w\s'А-Яа-яІіЇїЄєҐґ]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def tokens(s: str):
    t = normalize(s)
    out = []
    for w in t.split():
        if len(w) >= 3 and w not in UA_STOP:
            out.append(w)
    return out

def lexical_score(query: str, text: str):
    q = tokens(query)
    if not q:
        return 0
    tt = normalize(text)
    score = 0
    for w in q:
        if w in tt:
            score += 1
    return score

def search_chunks(query: str, topk=5):
    scored = []
    for i, ch in enumerate(chunks):
        s = lexical_score(query, ch.text or '')
        if s > 0:
            scored.append((s, i))
    scored.sort(reverse=True)
    return scored[:topk]

query = "реплікація ДНК"
hits = search_chunks(query, topk=5)
print('Query:', query)
print('Hits:', hits)
for rank, (s, idx) in enumerate(hits, 1):
    ch = chunks[idx]
    print('\n' + '='*60)
    print(f'Rank {rank} | score={s} | idx={idx}')
    print('Meta:', ch.meta)
    print('-'*60)
    print((ch.text or '')[:900])


## ✅ Очікуваний результат

- `Documents loaded` має бути > 0 (якщо в `data/raw` є файли)
- `Chunks created` теж > 0
- У виводі видно `meta` (source/start/end) і текст чанка
- В аналізі видно розподіл довжин і підозрілі чанки
- Lexical search дає кілька чанків, де реально є ключові слова
